In [6]:
from collections import defaultdict
import requests
import time, os

import numpy as np
import pandas as pd

import json, urllib3, string
from bs4 import BeautifulSoup

import pickle
from sqlalchemy import create_engine

from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from IPython.display import Image
import matplotlib.pylab as pylab

import re

import gender_guesser.detector as gender

import statsmodels.api as sm
from scipy.stats import chisquare
from sklearn.model_selection import train_test_split

## Import Data

### Import data from Bechdel Test Movie List

Description of data found here: http://bechdeltest.com/api/v1/doc

In [7]:
%%time
all_movies_response = requests.get('http://bechdeltest.com/api/v1/getAllMovies')

Wall time: 2.73 s


In [8]:
bechdel_movies = pd.DataFrame(all_movies_response.json())

In [9]:
bechdel_movies.sample(n=5)

,imdbid,title,id,year,rating
4279,0357507,Boogeyman,4968,2005,2
7708,1878841,"Darkness, The",7766,2016,2
5954,1723811,Shame,3511,2011,1
1512,0075860,Close Encounters of the Third Kind,3610,1977,1
6572,1699755,Peeples,4205,2013,3


In [10]:
bechdel_movies_mod = bechdel_movies.copy()
bechdel_movies_mod['imdbid_tt'] = 'tt'+ bechdel_movies_mod['imdbid']
bechdel_movies_mod.head()

,imdbid,title,id,year,rating,imdbid_tt
0,0392728,Roundhay Garden Scene,8040,1888,0,tt0392728
1,0000003,Pauvre Pierrot,5433,1892,0,tt0000003
2,0132134,"Execution of Mary, Queen of Scots, The",6200,1895,0,tt0132134
3,0000014,Tables Turned on the Gardener,5444,1895,0,tt0000014
4,0000131,Une nuit terrible,5406,1896,0,tt0000131


### Import data from IMDB

Description of data found here: https://www.imdb.com/interfaces/

This takes a long time to import

Download data (https://datasets.imdbws.com/), change filepath, and unzip here:    

In [5]:
# !gzip -d "C:\Users\wlee\Documents\Non Work Related\Metis\Project 3\imdb data\name.basics.tsv.gz"
# !gzip -d "C:\Users\wlee\Documents\Non Work Related\Metis\Project 3\imdb data\title.principals.tsv.gz"
# !gzip -d "C:\Users\wlee\Documents\Non Work Related\Metis\Project 3\imdb data\title.crew.tsv.gz"
# !gzip -d "C:\Users\wlee\Documents\Non Work Related\Metis\Project 3\imdb data\title.basics.tsv.gz"
# !gzip -d "C:\Users\wlee\Documents\Non Work Related\Metis\Project 3\imdb data\title.akas.tsv.gz"

In [12]:
data_dir = "C:\\Users\\wlee\\Documents\\Non Work Related\\Metis\\Project 3\\imdb data\\" #change this directory to where you unzipped the files
title_crew = pd.read_csv(data_dir + 'title.crew.tsv',sep='\t')
title_principals = pd.read_csv(data_dir + 'title.principals.tsv',sep='\t')
name_basics = pd.read_csv(data_dir + 'name.basics.tsv',sep='\t')
title_basics = pd.read_csv(data_dir + 'title.basics.tsv',sep='\t')
title_akas = pd.read_csv(data_dir + 'title.akas.tsv',sep='\t')

Describe Bechdel Data

In [13]:
print("Number of movies after 1950:",len(bechdel_movies[bechdel_movies['year']>=1950]))
print("DataFrame shape:",bechdel_movies_mod.shape)
print("Available columns:", bechdel_movies_mod.columns)

Number of movies after 1950: 8205
DataFrame shape: (8796, 6)
Available columns: Index(['imdbid', 'title', 'id', 'year', 'rating', 'imdbid_tt'], dtype='object')


In [16]:
from varname import nameof
datasets = [bechdel_movies,title_crew,title_basics,title_principals,title_akas,name_basics]
dataset_names = [nameof(bechdel_movies),nameof(title_crew),nameof(title_basics),nameof(title_principals),nameof(title_akas),nameof(name_basics)]
datasets_zip = list(zip(datasets,dataset_names))
print("List of data imported:")
for name in dataset_names:
    print(name)

List of data imported:
bechdel_movies
title_crew
title_basics
title_principals
title_akas
name_basics


## Data Processing

In [17]:
%time

merge = bechdel_movies_mod.merge(title_crew,how='inner',left_on='imdbid_tt',right_on='tconst')
# merge = merge.merge(title_principals,how='inner',left_on='imdbid_tt',right_on='tconst')
merge.head()

Wall time: 0 ns


,imdbid,title,id,year,rating,imdbid_tt,tconst,directors,writers
0,0392728,Roundhay Garden Scene,8040,1888,0,tt0392728,tt0392728,nm1284117,\N
1,0000003,Pauvre Pierrot,5433,1892,0,tt0000003,tt0000003,nm0721526,\N
2,0132134,"Execution of Mary, Queen of Scots, The",6200,1895,0,tt0132134,tt0132134,nm0163632,\N
3,0000014,Tables Turned on the Gardener,5444,1895,0,tt0000014,tt0000014,"nm0349785,nm0525910",\N
4,0000131,Une nuit terrible,5406,1896,0,tt0000131,tt0000131,nm0617588,\N


In [18]:
merge_principals = bechdel_movies_mod.merge(title_principals,how='inner',left_on='imdbid_tt',right_on='tconst')
principals_names = merge_principals.merge(name_basics[['nconst','primaryName']],how='inner',left_on='nconst',right_on='nconst')
principals_names['firstName'] = principals_names['primaryName'].str.extract('(^\S+) ')
principals_names = principals_names[(principals_names.category.isin(['actor','writer','actress','producer','director','editor']))]
principals_names.head()

,imdbid,title,id,year,rating,imdbid_tt,tconst,ordering,nconst,category,job,characters,primaryName,firstName
4,0392728,Roundhay Garden Scene,8040,1888,0,tt0392728,tt0392728,5,nm1284117,director,\N,\N,Louis Aimé Augustin Le Prince,Louis
5,0000003,Pauvre Pierrot,5433,1892,0,tt0000003,tt0000003,1,nm0721526,director,\N,\N,Émile Reynaud,Émile
6,0000003,Pauvre Pierrot,5433,1892,0,tt0000003,tt0000003,2,nm1770680,producer,producer,\N,Julien Pappé,Julien
8,0000003,Pauvre Pierrot,5433,1892,0,tt0000003,tt0000003,4,nm5442200,editor,\N,\N,Tamara Pappé,Tamara
9,0132134,"Execution of Mary, Queen of Scots, The",6200,1895,0,tt0132134,tt0132134,1,nm0858405,actor,\N,"[""Queen Mary""]",Robert Thomae,Robert


In [19]:
# merge[merge['imdbid_tt']=='tt0084745']

In [20]:
# merge_principals[merge_principals['imdbid_tt']=='tt0084745']

Process merged director data 

In [21]:
id_directors = pd.DataFrame(merge['directors'].str.split(',').tolist(),index=merge['imdbid_tt']).stack()
id_directors = id_directors.reset_index([0,'imdbid_tt'])
id_directors.columns = ['imdbid_tt','directors']

id_directors = id_directors.merge(name_basics[['nconst','primaryName']],how='inner',left_on='directors',right_on='nconst')
id_directors['firstName'] = id_directors['primaryName'].str.extract('(^\S+) ')
# id_directors['gender'] = id_directors['firstName'].apply(d.get_gender)

id_directors.head()

,imdbid_tt,directors,nconst,primaryName,firstName
0,tt0392728,nm1284117,nm1284117,Louis Aimé Augustin Le Prince,Louis
1,tt0000003,nm0721526,nm0721526,Émile Reynaud,Émile
2,tt0132134,nm0163632,nm0163632,Alfred Clark,Alfred
3,tt0000014,nm0349785,nm0349785,Alice Guy,Alice
4,tt0223341,nm0349785,nm0349785,Alice Guy,Alice


Process merged writer data

In [22]:
id_writers = pd.DataFrame(merge['writers'].str.split(',').tolist(),index=merge['imdbid_tt']).stack()
id_writers = id_writers.reset_index([0,'imdbid_tt'])
id_writers.columns = ['imdbid_tt','writers']

id_writers = id_writers.merge(name_basics[['nconst','primaryName']],how='inner',left_on='writers',right_on='nconst')
id_writers['firstName'] = id_writers['primaryName'].str.extract('(^\S+) ')
# id_writers['gender'] = id_writers['firstName'].apply(d.get_gender)

id_writers.head()

,imdbid_tt,writers,nconst,primaryName,firstName
0,tt0223341,nm0349785,nm0349785,Alice Guy,Alice
1,tt0000091,nm0617588,nm0617588,Georges Méliès,Georges
2,tt0000211,nm0617588,nm0617588,Georges Méliès,Georges
3,tt0131934,nm0617588,nm0617588,Georges Méliès,Georges
4,tt0000417,nm0617588,nm0617588,Georges Méliès,Georges


Generate dataset names

In [23]:
from varname import nameof
datasets = [merge,principals_names,id_directors,id_writers]
datasets_names = [nameof(merge),nameof(principals_names),nameof(id_directors),nameof(id_writers)]
datasets_zip = list(zip(datasets,datasets_names))
datasets_dict = {}
for item in datasets_zip:
    datasets_dict[item[1]] = item[0]

datasets_dict.keys()

dict_keys(['merge', 'principals_names', 'id_directors', 'id_writers'])

Pickle

In [24]:
with open('datasets_dict.pickle', 'wb') as f:
    pickle.dump(datasets_dict, f, protocol=4)